In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pywt
import os
import librosa 
from glob import glob
from sklearn.model_selection import train_test_split
import tensorflow as tf
from keras.models import Model
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import SGD, Adam, Nadam
from keras import regularizers
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, precision_score, recall_score, ConfusionMatrixDisplay
from sklearn.model_selection import RandomizedSearchCV, train_test_split
from scipy.stats import randint

In [2]:
normal = sorted(glob(os.path.join("", "normal", "*.wav")))
abnormal = sorted(glob(os.path.join("", "abnormal", "*.wav")))

In [3]:
x=abnormal+normal

In [4]:
y_t= [[0]]*408
n = [[1]]*533
y_t = y_t + n
y = np.array(y_t)

In [5]:
rfy_t= [0]*408
rfn = [1]*533
rfy_t = rfy_t + rfn
rfy = np.array(rfy_t)

In [6]:
x_train, x_test = train_test_split(x, test_size=0.3, random_state=42)
y_train, y_test = train_test_split(y, test_size=0.3, random_state=42)
rfy_train, rfy_test = train_test_split(rfy, test_size=0.3, random_state=42)

In [7]:
def load_data(path):
    x_t = []
    for i in path:
        data, sr = librosa.load(i, sr=8000)
        data=data[:147000]
        new_sample_rate = 1000
        resampled_data = librosa.resample(data, orig_sr=sr, target_sr=new_sample_rate)
        x_t.append(resampled_data)
    return x_t

In [8]:
X_train=load_data(x_train)
X_test=load_data(x_test)

In [9]:
def wavelet_transform(data):
    for i in range(len(data)):
        coeffs = pywt.wavedec(data[i], wavelet='haar', level=5)
        new = np.zeros(0)
        for j in coeffs:
            new = np.concatenate((new, j), axis=-1)
        data[i] = new

In [10]:
wavelet_transform(X_train)
wavelet_transform(X_test)

In [11]:
X_train=np.array(X_train)
X_test=np.array(X_test)

In [20]:
model1 = tf.keras.models.Sequential([
    tf.keras.layers.Dense(256, activation='relu', input_shape=(18378,)),
    tf.keras.layers.Dropout(0.4),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dropout(0.7),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dropout(0.3),
    tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

opt1 = tf.keras.optimizers.Adam(learning_rate=0.001)
loss_fn1 = tf.keras.losses.BinaryCrossentropy()
model1.compile(optimizer = opt1,
              loss = loss_fn1,
              metrics = ['accuracy'])
model1.fit(X_train, y_train, epochs=10)

_, test_accuracy = model1.evaluate(X_test, y_test, verbose=0)
print(test_accuracy)


Epoch 1/10
21/21 [==============================] - 3s 69ms/step - loss: 0.7035 - accuracy: 0.5760
Epoch 2/10
21/21 [==============================] - 1s 65ms/step - loss: 0.8136 - accuracy: 0.5745
Epoch 3/10
21/21 [==============================] - 1s 69ms/step - loss: 0.7953 - accuracy: 0.5502
Epoch 4/10
21/21 [==============================] - 2s 79ms/step - loss: 0.7934 - accuracy: 0.5927
Epoch 5/10
21/21 [==============================] - 1s 67ms/step - loss: 0.7704 - accuracy: 0.6109
Epoch 6/10
21/21 [==============================] - 1s 66ms/step - loss: 0.7188 - accuracy: 0.6459
Epoch 7/10
21/21 [==============================] - 1s 66ms/step - loss: 0.6377 - accuracy: 0.6778
Epoch 8/10
21/21 [==============================] - 1s 63ms/step - loss: 0.6319 - accuracy: 0.7204
Epoch 9/10
21/21 [==============================] - 1s 62ms/step - loss: 0.6113 - accuracy: 0.7416
Epoch 10/10
21/21 [==============================] - 1s 64ms/step - loss: 0.4948 - accuracy: 0.8252
0.4946996

In [42]:
prediction = model1.predict(X_test[22].reshape(1,18378))  # a zero
print(f" predicting a one: {prediction}")
print(f"y_hat : {y_test[22]}")
prediction = model1.predict(X_test[20].reshape(1,18378))  # a one
print(f" predicting a zero:  {prediction}")
print(f"y_hat : {y_test[20]}")

1/1 [==============================] - 0s 49ms/step
 predicting a one: [[0.5251371]]
y_hat : [1]
1/1 [==============================] - 0s 21ms/step
 predicting a zero:  [[0.5066145]]
y_hat : [0]


In [24]:
model2 = tf.keras.models.Sequential([
    tf.keras.layers.Dense(256, activation='relu', input_shape=(18378,), kernel_regularizer=regularizers.l2(l=0.01)),
    tf.keras.layers.Dropout(0.4),
    tf.keras.layers.Dense(128, activation='relu', kernel_regularizer=regularizers.l2(l=0.001)),
    tf.keras.layers.Dropout(0.7),
    tf.keras.layers.Dense(64, activation='relu', kernel_regularizer=regularizers.l2(l=0.1)),
    tf.keras.layers.Dropout(0.3),
    tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

opt2 = tf.keras.optimizers.Adam(learning_rate=0.001)
loss_fn2 = tf.keras.losses.BinaryCrossentropy()
model2.compile(optimizer = opt2,
              loss = loss_fn2,
              metrics = ['accuracy'])
model2.fit(X_train, y_train, epochs=10)

_, test_accuracy = model2.evaluate(X_test, y_test, verbose=0)
print(test_accuracy)


Epoch 1/10
21/21 [==============================] - 3s 68ms/step - loss: 11.0632 - accuracy: 0.5106
Epoch 2/10
21/21 [==============================] - 1s 70ms/step - loss: 7.5211 - accuracy: 0.5380
Epoch 3/10
21/21 [==============================] - 2s 78ms/step - loss: 5.5406 - accuracy: 0.5365
Epoch 4/10
21/21 [==============================] - 2s 81ms/step - loss: 4.0915 - accuracy: 0.5745
Epoch 5/10
21/21 [==============================] - 2s 82ms/step - loss: 3.0995 - accuracy: 0.5775
Epoch 6/10
21/21 [==============================] - 2s 72ms/step - loss: 2.3594 - accuracy: 0.5988
Epoch 7/10
21/21 [==============================] - 2s 73ms/step - loss: 1.8602 - accuracy: 0.6064
Epoch 8/10
21/21 [==============================] - 2s 72ms/step - loss: 1.5107 - accuracy: 0.6216
Epoch 9/10
21/21 [==============================] - 2s 73ms/step - loss: 1.2870 - accuracy: 0.6565
Epoch 10/10
21/21 [==============================] - 2s 73ms/step - loss: 1.1138 - accuracy: 0.7872
0.477031

In [41]:
prediction = model2.predict(X_test[22].reshape(1,18378))  # a zero
print(f" predicting a one: {prediction}")
print(f"y_hat : {y_test[22]}")
prediction = model2.predict(X_test[20].reshape(1,18378))  # a one
print(f" predicting a zero:  {prediction}")
print(f"y_hat : {y_test[20]}")

1/1 [==============================] - 0s 54ms/step
 predicting a one: [[0.5262837]]
y_hat : [1]
1/1 [==============================] - 0s 17ms/step
 predicting a zero:  [[0.56044644]]
y_hat : [0]


In [36]:
model3 = tf.keras.models.Sequential([
    tf.keras.layers.Conv1D(128, 16, input_shape=(18378,1)),
    tf.keras.layers.MaxPool1D(),
    tf.keras.layers.Conv1D(64, 16),
    tf.keras.layers.MaxPool1D(),
    tf.keras.layers.Conv1D(32, 4),
    tf.keras.layers.MaxPool1D(),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

opt3 = tf.keras.optimizers.Adam(learning_rate=0.001)
loss_fn3 = tf.keras.losses.BinaryCrossentropy()
model3.compile(optimizer = opt3,
              loss = loss_fn3,
              metrics = ['accuracy'])
model3.fit(X_train, y_train, epochs=10)

_, test_accuracy = model3.evaluate(X_test, y_test, verbose=0)
print(test_accuracy)


Epoch 1/10
21/21 [==============================] - 46s 2s/step - loss: 0.6883 - accuracy: 0.5578
Epoch 2/10
21/21 [==============================] - 51s 2s/step - loss: 0.6026 - accuracy: 0.6793
Epoch 3/10
21/21 [==============================] - 53s 3s/step - loss: 0.5263 - accuracy: 0.7325
Epoch 4/10
21/21 [==============================] - 52s 2s/step - loss: 0.4194 - accuracy: 0.7948
Epoch 5/10
21/21 [==============================] - 52s 2s/step - loss: 0.2984 - accuracy: 0.8663
Epoch 6/10
21/21 [==============================] - 52s 2s/step - loss: 0.2173 - accuracy: 0.9043
Epoch 7/10
21/21 [==============================] - 53s 2s/step - loss: 0.1893 - accuracy: 0.9240
Epoch 8/10
21/21 [==============================] - 52s 2s/step - loss: 0.2107 - accuracy: 0.8921
Epoch 9/10
21/21 [==============================] - 52s 2s/step - loss: 0.1262 - accuracy: 0.9620
Epoch 10/10
21/21 [==============================] - 52s 2s/step - loss: 0.0815 - accuracy: 0.9772
0.685512363910675


In [39]:
prediction = model3.predict(X_test[22].reshape(1,18378))  # a zero
print(f" predicting a one: {prediction}")
print(f"y_hat : {y_test[22]}")
prediction = model3.predict(X_test[20].reshape(1,18378))  # a one
print(f" predicting a zero:  {prediction}")
print(f"y_hat : {y_test[20]}")

1/1 [==============================] - 0s 62ms/step
 predicting a one: [[0.9822676]]
y_hat : [1]
1/1 [==============================] - 0s 51ms/step
 predicting a zero:  [[0.11430689]]
y_hat : [0]


In [34]:
model4 = tf.keras.models.Sequential([
    tf.keras.layers.Conv1D(128, 16, input_shape=(18378,1), kernel_regularizer=regularizers.l2(l=0.01)),
    tf.keras.layers.MaxPool1D(),
    tf.keras.layers.Dropout(0.6),
    tf.keras.layers.Conv1D(32, 16,kernel_regularizer=regularizers.l2(l=0.01)),
    tf.keras.layers.MaxPool1D(),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Conv1D(8, 4, kernel_regularizer=regularizers.l2(l=0.01)),
    tf.keras.layers.MaxPool1D(),
    tf.keras.layers.Dropout(0.4),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

opt4 = tf.keras.optimizers.Adam(learning_rate=0.001)
loss_fn4 = tf.keras.losses.BinaryCrossentropy()
model4.compile(optimizer = opt4,
              loss = loss_fn4,
              metrics = ['accuracy'])
model4.fit(X_train, y_train, epochs=15)

_, test_accuracy = model4.evaluate(X_test, y_test, verbose=0)
print(test_accuracy)


Epoch 1/15
21/21 [==============================] - 51s 2s/step - loss: 1.2326 - accuracy: 0.5471
Epoch 2/15
21/21 [==============================] - 46s 2s/step - loss: 1.0140 - accuracy: 0.5942
Epoch 3/15
21/21 [==============================] - 46s 2s/step - loss: 0.8922 - accuracy: 0.6018
Epoch 4/15
21/21 [==============================] - 45s 2s/step - loss: 0.7928 - accuracy: 0.6489
Epoch 5/15
21/21 [==============================] - 46s 2s/step - loss: 0.7136 - accuracy: 0.7158
Epoch 6/15
21/21 [==============================] - 47s 2s/step - loss: 0.6267 - accuracy: 0.7371
Epoch 7/15
21/21 [==============================] - 46s 2s/step - loss: 0.5764 - accuracy: 0.7675
Epoch 8/15
21/21 [==============================] - 50s 2s/step - loss: 0.5472 - accuracy: 0.7842
Epoch 9/15
21/21 [==============================] - 47s 2s/step - loss: 0.5212 - accuracy: 0.8040
Epoch 10/15
21/21 [==============================] - 46s 2s/step - loss: 0.5223 - accuracy: 0.7766
Epoch 11/15
21/21 [

In [40]:
prediction = model4.predict(X_test[22].reshape(1,18378))  # a zero
print(f" predicting a one: {prediction}")
print(f"y_hat : {y_test[22]}")
prediction = model4.predict(X_test[20].reshape(1,18378))  # a one
print(f" predicting a zero:  {prediction}")
print(f"y_hat : {y_test[20]}")

1/1 [==============================] - 0s 51ms/step
 predicting a one: [[0.83666426]]
y_hat : [1]
1/1 [==============================] - 0s 24ms/step
 predicting a zero:  [[0.40125242]]
y_hat : [0]


In [16]:
param_dist = {'n_estimators': randint(500,5000),
              'max_depth': randint(1,500)}

# Create a random forest classifier
rf = RandomForestClassifier()

# Use random search to find the best hyperparameters
rand_search = RandomizedSearchCV(rf, 
                                 param_distributions = param_dist, 
                                 n_iter=5, 
                                 cv=5)

# Fit the random search object to the data
rand_search.fit(X_train, rfy_train)

RandomizedSearchCV(cv=5, estimator=RandomForestClassifier(), n_iter=5,
                   param_distributions={'max_depth': <scipy.stats._distn_infrastructure.rv_discrete_frozen object at 0x00000285F952A250>,
                                        'n_estimators': <scipy.stats._distn_infrastructure.rv_discrete_frozen object at 0x00000285F944B8E0>})

In [14]:
# Create a variable for the best model
best_rf = rand_search.best_estimator_

# Print the best hyperparameters
print('Best hyperparameters:',  rand_search.best_params_)

Best hyperparameters: {'max_depth': 74, 'n_estimators': 905}


In [19]:
rf = RandomForestClassifier(n_estimators=best_rf.n_estimators, max_depth=best_rf.max_depth)
rf.fit(X_train, rfy_train)
y_pred = rf.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

Accuracy: 0.696113074204947
